[View in Colaboratory](https://colab.research.google.com/github/NikhilNandoskar/Deep_Learning/blob/master/CIFAR_10_DNN.ipynb)

In [0]:
!pip3 install tqdm

from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm 
#import unittest 
from unittest import *
import tarfile

cifar10_dataset_folder_path = 'cifar-10-batches-py'

class DownloadProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

""" 
    check if the data (zip) file is already downloaded
    if not, download it from "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz" and save as cifar-10-python.tar.gz
"""
if not isfile('cifar-10-python.tar.gz'):
    with DownloadProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            'cifar-10-python.tar.gz',
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open('cifar-10-python.tar.gz') as tar:
        tar.extractall()
        tar.close()


#test.test_folder_path(cifar10_dataset_folder_path)

    100% |████████████████████████████████| 51kB 2.4MB/s 


CIFAR-10 Dataset: 171MB [00:17, 9.94MB/s]                           


In [0]:
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def unpickle(file):
  import pickle
  with open(file, 'rb') as fo:
    cif_dict = pickle.load(fo, encoding = 'bytes')
  return cif_dict

In [0]:
dirs = ['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']
all_data = [0,1,2,3,4,5,6]

In [0]:
for i, direc in zip(all_data, dirs):
  all_data[i] = unpickle(cifar10_dataset_folder_path+direc)

FileNotFoundError: ignored

In [0]:
def one_hot_encoding(vec, value = 10):
  out = np.zeros((len(vec), value))
  out[range(len(vec), vec)] = 1
  return out

In [0]:
class CifarHelper():
    
    def __init__(self):
        self.i = 0
        
        self.all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
        self.test_batch = [test_batch]
        
        self.training_images = None
        self.training_labels = None
        
        self.test_images = None
        self.test_labels = None
    
    def set_up_images(self):
        
        print("Setting Up Training Images and Labels")
        
        self.training_images = np.vstack([d[batch['data']] for d in self.all_train_batches])
        train_len = len(self.training_images)
        
        self.training_images = self.training_images.reshape(train_len,3,32,32).transpose(0,2,3,1)/255
        self.training_labels = one_hot_encode(np.hstack([d[batch['labels']] for d in self.all_train_batches]), 10)
        
        print("Setting Up Test Images and Labels")
        
        self.test_images = np.vstack([d[batch['data']] for d in self.test_batch])
        test_len = len(self.test_images)
        
        self.test_images = self.test_images.reshape(test_len,3,32,32).transpose(0,2,3,1)/255
        self.test_labels = one_hot_encode(np.hstack([d[batch['labels']] for d in self.test_batch]), 10)

        
    def next_batch(self, batch_size):
        x = self.training_images[self.i:self.i+batch_size].reshape(100,32,32,3)
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_images)
        return x, y

In [0]:
# Before Your tf.Session run these two lines
ch = CifarHelper()
ch.set_up_images()

# During your session to grab the next batch use this line
# (Just like we did for mnist.train.next_batch)
# batch = ch.next_batch(100)

Setting Up Training Images and Labels


NameError: ignored

In [0]:
#TensoeFlow 
import tensorflow as tf

In [0]:
x = tf.placeholder(tf.float32, shape = [None, 32, 32, 3], name = 'input_x')
y_truth = tf.placeholder(tf.float32, shape = [None, 10], name = 'output_y')
hold_prob = tf.placeholder(tf.float32, name = 'hold_prob')

In [0]:
def max_pool(x):
  return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [0]:
#Creating Layers

def conv_net(x, hold_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 64], mean=0, stddev=0.1))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.1))
    conv3_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 128, 256], mean=0, stddev=0.1))
    conv4_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 256, 512], mean=0, stddev=0.1))


    
    conv_1 = tf.nn.conv2d(x, conv1_filter, strides = [1,1,1,1], padding = 'SAME')
    conv_1 = tf.nn.relu(conv_1)
    conv_1_pooling = max_pool(conv_1)
    conv_1_bn = tf.layers.batch_normalization(conv_1_pooling)

    conv_2 = tf.nn.conv2d(conv_1_bn, conv2_filter, strides = [1,1,1,1], padding = 'SAME')
    conv_2 = tf.nn.relu(conv_2)
    conv_2_pooling = max_pool(conv_2)
    conv_2_bn = tf.layers.batch_normalization(conv_2_pooling)

    conv_3 = tf.nn.conv2d(conv_2_bn, conv3_filter, strides = [1,1,1,1], padding = 'SAME')
    conv_3 = tf.nn.relu(conv_3) 
    conv_3_pooling = max_pool(conv_3)
    conv_3_bn = tf.layers.batch_normalization(conv_3_pooling)

    conv_4 = tf.nn.conv2d(conv_3_bn, conv4_filter, strides = [1,1,1,1], padding = 'SAME')
    conv_4 = tf.nn.relu(conv_4)
    conv_4_pooling = max_pool(conv_4)
    conv_4_bn = tf.layers.batch_normalization(conv_4_pooling)

    flat = tf.contrib.layers.flatten(conv_4_bn)

    full_lay_1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    full_lay_1 = tf.nn.dropout(full_lay_1, keep_prob = hold_prob)
    full_lay_1 = tf.layers.batch_normalization(full_lay_1)


    full_lay_2 = tf.contrib.layers.fully_connected(inputs=full_lay_1, num_outputs=256, activation_fn=tf.nn.relu)
    full_lay_2 = tf.nn.dropout(full_lay_2, keep_prob = hold_prob)
    full_lay_2 = tf.layers.batch_normalization(full_lay_2)


    full_lay_3 = tf.contrib.layers.fully_connected(inputs=full_lay_2, num_outputs=512, activation_fn=tf.nn.relu)
    full_lay_3 = tf.nn.dropout(full_lay_3, keep_prob = hold_prob)
    full_lay_3 = tf.layers.batch_normalization(full_lay_3)


    full_lay_4 = tf.contrib.layers.fully_connected(inputs=full_lay_3, num_outputs=1024, activation_fn=tf.nn.relu)
    full_lay_4 = tf.nn.dropout(full_lay_4, keep_prob = hold_prob)
    full_lay_4 = tf.layers.batch_normalization(full_lay_4)

    out = tf.contrib.layers.fully_connected(inputs=full_lay_4, num_outputs=10, activation_fn=None)
    return out

In [0]:
y_pred = conv_net(x, hold_prob)
#model = tf.identity(logits, name = 'logits')

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_truth ))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(5000):
        batch = ch.next_batch(100)
        sess.run(train, feed_dict={x: batch[0], y_true: batch[1], hold_prob: 0.7})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_truth,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:ch.test_images,y_true:ch.test_labels,hold_prob:1.0}))
            print('\n')

NameError: ignored